# CNN (Convolutional Neural Network)

- 참고
    - Image classification with deep convolutional neural networks (Khan et al., 2020)
    - ImageNet Classification with Deep Convolutional Neural Networks (Krizhevsky et al., 2012)

- Convolutional Neural Network(CNN)은 컴퓨터 비전 및 이미지 처리와 관련된 여러 대회에서 우수한 성능을 보인 특별한 유형의 신경망
- CNN은 이미지 내의 특징을 추출하고 이를 이용해 이미지를 분류하거나 객체를 검출하는 등의 작업을 수행할 수 있음
- CNN은 입력 이미지를 여러 개의 레이어로 처리하며, 각 레이어에서는 입력 이미지와 필터를 합성곱하여 특징 맵(feature map)을 생성
- 이때 필터는 이미지에서 특정한 패턴을 찾아내기 위한 가중치 값으로, 학습 과정에서 자동으로 조정됨
- 레이어를 여러 번 거침으로써, 점점 더 추상적인 특징을 추출할 수 있음

- ImageNetClassification with Deep Convolutional Neural Networks에서는 8개의 레이어로 이루어진 CNN 아키텍처인 AlexNet을 제안함
- 이 아키텍처는 풀링 레이어, 컨볼루션 레이어, 활성화 함수, 드롭아웃 레이어 등 다양한 컴포넌트로 구성됨
- 또한, 큰 데이터셋과 병렬 컴퓨팅을 이용하여 CNN의 학습을 가속화하는 방법도 제안하고 있음

- AlexNet은 기존의 신경망 아키텍처에 비해 몇 가지 차이점이 있음
    1. 더 많은 학습 데이터 사용: AlexNet은 120만 개 이상의 이미지를 사용하여 학습
    2. GPU를 사용한 병렬 처리: AlexNet은 2개의 GPU를 사용하여 효율적으로 학습할 수 있도록 설계
    3. ReLU 활성화 함수 사용: 기존의 신경망에서는 sigmoid 함수나 tanh 함수를 사용했지만, AlexNet에서는 ReLU(Rectified Linear Unit) 함수를 사용하여 학습 속도를 높임
    4. Local Response Normalization: 이전 레이어에서 활성화된 뉴런들이 다음 레이어에서 높은 활성화 값을 가지도록 정규화를 적용
    5. Dropout: 학습 데이터에서 무작위로 뉴런을 제거하면서 학습하는 방법으로, 오버피팅(overfitting)을 방지하는 데 효과적
- 논문에서는 또한 maxpooling과 conv2d도 자세히 설명하고 있음
    - Maxpooling은 이미지의 크기를 줄이고, 불필요한 정보를 제거하는 데 사용됨
    - Conv2d는 이미지에서 특징을 추출하는 데 사용되는 컨볼루션 필터를 적용하는 방법

# 실습
- 캐글(Kaggle)의 개 고양이 이미지 분류 대회의 이미지 데이터셋을 이용한 CNN 이미지 분류

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")
# 실행에 필요한 메시지 외 경고가 출력되지 않게 해줌
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from tensorflow import keras

from sklearn.model_selection import train_test_split

## 전처리

In [ ]:
imageDataGenerator = ImageDataGenerator(
    rescale=1./255,                  # 고정 이미지 - 픽셀값을 0~255에서 0~1범위로 변경
    shear_range=0.2,                 # 기울기 범위
    zoom_range=0.2,                  # 확대 범위
    horizontal_flip=True,           # 상하반전
    validation_split=0.1            # 배치비율(예: 0.3, 0.2, 0.25 등등)
)
imageDataGenerator_test = ImageDataGenerator(rescale=1./255)

train_dataset = imageDataGenerator.flow_from_directory(
    "./data/train/",
    target_size=(128,128),
    batch_size=100,
    subset="training",
    class_mode="binary"
)

val_dataset = imageDataGenerator.flow_from_directory(
    "./data/train/",
    target_size=(128,128),
    batch_size=100,
    subset="validation",
    class_mode="binary"
)

test_dataset = imageDataGenerator_test.flow_from_directory(
    "./data/test/",
    target_size=(128,128),
    batch_size=100,
    shuffle=False
)

- 개 이미지 12,500개, 고양이 이미지 12,500개를 train/validation/test 용으로 분리하고, tensorflow의 ImageDataGenerator를 사용할 것이기 때문에 각 라벨(cat,dog)를 폴더별로 따로 분리

## 모델 생성 및 평가

In [128]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 16, 16, 32)     

In [129]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-3model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [130]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [131]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("model.h5")

Epoch 1/100
225/225 [==============================] - 201s 885ms/step - loss: 0.6478 - accuracy: 0.6076 - val_loss: 0.5838 - val_accuracy: 0.6764
Epoch 2/100
225/225 [==============================] - 186s 826ms/step - loss: 0.5646 - accuracy: 0.7076 - val_loss: 0.5241 - val_accuracy: 0.7272
Epoch 3/100
225/225 [==============================] - 185s 822ms/step - loss: 0.5202 - accuracy: 0.7446 - val_loss: 0.4835 - val_accuracy: 0.7792
Epoch 4/100
225/225 [==============================] - 185s 824ms/step - loss: 0.4873 - accuracy: 0.7692 - val_loss: 0.4540 - val_accuracy: 0.7804
Epoch 5/100
225/225 [==============================] - 183s 814ms/step - loss: 0.4716 - accuracy: 0.7815 - val_loss: 0.4525 - val_accuracy: 0.7968
Epoch 6/100
225/225 [==============================] - 186s 826ms/step - loss: 0.4450 - accuracy: 0.7982 - val_loss: 0.4028 - val_accuracy: 0.8128
Epoch 7/100
225/225 [==============================] - 189s 840ms/step - loss: 0.4263 - accuracy: 0.8103 - val_loss: 0

In [132]:
pred = model.predict(test_dataset)

125/125 [==============================] - 37s 291ms/step
